In [1]:
# Required packages

!pip install BeautifulSoup4 --break-system-packages
!pip install requests --break-system-packages

In [2]:
import requests
from bs4 import BeautifulSoup, Comment
import re
import time

In [3]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"
}

web_session = requests.Session()
web_session.headers.update(headers)

In [4]:
class Queue:
    def __init__(self):
        self.items = []

    def is_empty(self):
        return len(self.items) == 0

    def enqueue(self, item):
        self.items.insert(0, item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)

In [5]:
def get_all_page_urls(zip_code):
    url_queue = Queue()
    
    base_url = f"https://www.redfin.com/zipcode/{zip_code}/"
    
    page_regex = re.compile("Viewing page \\d+ of (\\d+)")

    request = web_session.get(base_url)
    if request.status_code != 200:
        return url_queue
    
    html_body = request.text
    page = BeautifulSoup(html_body)
    element = page.find("span", attrs={"data-rf-test-name": "download-and-save-page-number-text"})
    
    regex_result = page_regex.search(element.get_text())
    if regex_result == None:
        return url_queue

    url_queue.enqueue(base_url)
    
    max_page = int(regex_result.group(1))

    if max_page > 1:
        for i in range(2, max_page + 1):
            url_queue.enqueue(f"{base_url}page-{i}")

    return url_queue

In [6]:
def get_all_listings(zip_code):
    page_url_queue = get_all_page_urls(zip_code)
    number_regex = re.compile("^(?:\\d|\\.)+")
    
    all_listings = []
    
    while not page_url_queue.is_empty():
        url = page_url_queue.dequeue()
        html_body = web_session.get(url).text
        page = BeautifulSoup(html_body)

        for listing_card in page.find_all("div", class_="HomeCardContainer"):
            if listing_card.get("aria-label") and listing_card["aria-label"] == "Advertisement":
                continue

            listing_href = listing_card.find("a", href=True, class_="bp-InteractiveHomecard")["href"]
            price = listing_card.find("span", class_="bp-Homecard__Price--value").text
            price = float(price[1:].replace(",", ""))
            
            address = listing_card.find("div", class_="bp-Homecard__Address").string
            
            num_beds = listing_card.find("span", class_="bp-Homecard__Stats--beds").string.split(" ")[0]
            if num_beds == "—":
                continue
            else:
                num_beds = float(num_beds)

            num_baths = listing_card.find("span", class_="bp-Homecard__Stats--baths").string.split(" ")[0]
            if num_baths == "—":
                continue
            else:
                num_baths = float(num_baths)

            square_footage = listing_card.find("span", class_="bp-Homecard__Stats--sqft").find_all(string=True)[0]
            if square_footage in ["—", "sq ft"]:
                continue
            else:
                square_footage = float(square_footage.replace(",", ""))
            
            try:
                contact_info = listing_card.find("div", class_="bp-Homecard__Attribution").string
            except AttributeError:
                contact_info = "See listing via URL"

            listing_info = {
                "listing_url": f"https://redfin.com{listing_href}",
                "price": price,
                "address": address,
                "num_beds": num_beds,
                "num_baths": num_baths,
                "square_footage": square_footage,
                "contact_info": contact_info
            }

            all_listings.append(listing_info)
        time.sleep(1)

    return all_listings

In [7]:
def filter_listings(all_listings, min_price=None, max_price=None, min_beds=None, max_beds=None, min_baths=None, max_baths=None, min_sqft=None, max_sqft=None):
    filtered_listings = []
    for listing in all_listings:
        conditions = [
            (min_price is None or listing["price"] >= min_price),
            (max_price is None or listing["price"] <= max_price),
            (min_beds is None or listing["num_beds"] >= min_beds),
            (max_beds is None or listing["num_beds"] <= max_beds),
            (min_baths is None or listing["num_baths"] >= min_baths),
            (max_baths is None or listing["num_baths"] <= max_baths),
            (min_sqft is None or listing["square_footage"] >= min_sqft),
            (max_sqft is None or listing["square_footage"] <= max_sqft)
        ]

        if all(conditions):
            filtered_listings.append(listing)

    return filtered_listings

In [8]:
listings_example = get_all_listings("19104")
print(filter_listings(listings_example, max_price=150000))

[{'listing_url': 'https://redfin.com/PA/Philadelphia/706-N-Shedwick-St-19104/home/38669915', 'price': 139900.0, 'address': '706 N Shedwick St, Philadelphia, PA 19104', 'num_beds': 2.0, 'num_baths': 1.0, 'square_footage': 840.0, 'contact_info': '(215) 921-9053'}, {'listing_url': 'https://redfin.com/PA/Philadelphia/3833-Wyalusing-Ave-19104/home/38673323', 'price': 95000.0, 'address': '3833 Wyalusing Ave, Philadelphia, PA 19104', 'num_beds': 3.0, 'num_baths': 1.0, 'square_footage': 1010.0, 'contact_info': '(610) 520-0100'}, {'listing_url': 'https://redfin.com/PA/Philadelphia/3976-Wyalusing-Ave-19104/home/38673332', 'price': 150000.0, 'address': '3976 Wyalusing Ave, Philadelphia, PA 19104', 'num_beds': 3.0, 'num_baths': 1.0, 'square_footage': 876.0, 'contact_info': '(610) 679-9170'}, {'listing_url': 'https://redfin.com/PA/Philadelphia/3854-Reno-St-19104/home/38670012', 'price': 99999.0, 'address': '3854 Reno St, Philadelphia, PA 19104', 'num_beds': 4.0, 'num_baths': 1.0, 'square_footage': 